In [3]:
from pytrends.request import TrendReq 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
from pprint import pprint
import time

from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error 



In [187]:
cases = pd.read_csv("/Users/chengyichen/Desktop/pro_cases.csv",sep = ',') 
region = pd.read_csv("/Users/chengyichen/Desktop/correspondence_table.csv") 
correspondence = pd.read_excel('/Users/chengyichen/Desktop/region_table.xlsx') 

In [188]:
cases = cases.drop(columns = ['Unnamed: 0'])
cases = cases.applymap(lambda x: x.replace(",", "") if isinstance(x, str) else x)
cases = cases.rename(columns={"semaine": "week"})

cases

,dep,week,pop,P,T,Ti,Tp,Td,cl_age90
0,1,2020-S20,65695500,2300,140700,350,163,21417,0
1,1,2020-S21,65695500,4100,232200,624,177,35345,0
2,1,2020-S22,65695500,1900,218700,289,087,33290,0
3,2,2020-S20,52605000,5200,138000,988,377,26233,0
4,2,2020-S21,52605000,4100,131100,779,313,24922,0
...,...,...,...,...,...,...,...,...,...
11226,974,2022-S22,85995900,198300,1224000,23059,1620,142332,0
11227,976,2022-S22,27947100,9400,229400,3363,410,82084,0
11228,978,2022-S22,3533400,7600,111000,21509,685,314145,0
11229,975,2022-S22,599700,100,3400,1668,294,56695,0


In [329]:
region.head()
region = region.rename(columns={"Code_region_google": "Code"})
region

,Departement,Departement_label,Region_google,Code
0,1,Ain,Rhone-Alpes,V
1,2,Aisne,Picardie,S
2,3,Allier,Auvergne,C
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Cote d'Azur,U
4,5,Hautes-Alpes,Provence-Alpes-Cote d'Azur,U
...,...,...,...,...
96,971,Guadeloupe,NaN,No data retrieved in Google Trends.
97,972,Martinique,NaN,No data retrieved in Google Trends.
98,973,Guyane,NaN,No data retrieved in Google Trends.
99,974,La Reunion,NaN,No data retrieved in Google Trends.


In [340]:
correspondence

,Code
0,V
1,S
2,C
3,U
7,G
8,N
10,K
13,P
15,T
17,F


In [191]:
correspondence = correspondence.drop(columns=['dep']).drop_duplicates()

In [330]:
list_dep = []
for i in region['Departement']:
    list_dep.append(str(i))

region_table = pd.DataFrame({'dep':list_dep,'Code': region['Code']})
dep_table = pd.merge(cases,region_table,on = 'dep',how = 'left')
dep_table = dep_table.drop(columns = ['T'])
dep_table = dep_table.dropna()
dep_table

#    "P" （"Nombre de personnes positives détectées"）
#    "T" （"Nombre total de personnes testées"）
#    "Ti" （"Nombre de personnes hospitalisées"）
#   "Tp" （"Nombre de personnes en réanimation"）
#    "Td" （"Nombre de personnes décédées"）

,dep,week,pop,P,Ti,Tp,Td,cl_age90,Code
0,1,2020-S20,65695500,2300,350,163,21417,0,V
1,1,2020-S21,65695500,4100,624,177,35345,0,V
2,1,2020-S22,65695500,1900,289,087,33290,0,V
3,2,2020-S20,52605000,5200,988,377,26233,0,S
4,2,2020-S21,52605000,4100,779,313,24922,0,S
...,...,...,...,...,...,...,...,...,...
11223,971,2022-S22,37687900,216000,57313,1800,318458,0,No data retrieved in Google Trends.
11224,972,2022-S22,35874900,823400,229520,3679,623807,0,No data retrieved in Google Trends.
11225,973,2022-S22,29069100,85800,29516,1840,160376,0,No data retrieved in Google Trends.
11226,974,2022-S22,85995900,198300,23059,1620,142332,0,No data retrieved in Google Trends.


In [192]:
week_dep = pd.DataFrame(dep_table.groupby(['dep','week','Code'])['P','pop'].sum()) 
week_dep

/var/folders/w3/v_7zxn855qd1kj4wqy68swd80000gn/T/ipykernel_62955/1135654401.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  week_dep = pd.DataFrame(dep_table.groupby(['dep','week','Code'])['P','pop'].sum())


P       pop
dep week     Code                                                
1   2020-S20 V                                     2300  65695500
    2020-S21 V                                     4100  65695500
    2020-S22 V                                     1900  65695500
    2020-S23 V                                     1800  65695500
    2020-S24 V                                     1500  65695500
...                                                 ...       ...
976 2022-S18 No data retrieved in Google Trends.   7200  27947100
    2022-S19 No data retrieved in Google Trends.   6400  27947100
    2022-S20 No data retrieved in Google Trends.   5800  27947100
    2022-S21 No data retrieved in Google Trends.   7600  27947100
    2022-S22 No data retrieved in Google Trends.   9400  27947100

[10908 rows x 2 columns]

In [207]:
week_dep['P']= week_dep['P'].astype(int)
week_dep['pop'] = week_dep['pop'].astype(int)
week_dep = week_dep.reset_index()
week_dep = pd.DataFrame(week_dep.groupby(['Code','week'])['P','pop'].sum())
week_dep = week_dep.reset_index()
week_dep['incidence'] = (week_dep['P']/week_dep['pop']) * 100000

week_dep 


/var/folders/w3/v_7zxn855qd1kj4wqy68swd80000gn/T/ipykernel_62955/1982645843.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  week_dep = pd.DataFrame(week_dep.groupby(['Code','week'])['P','pop'].sum())


,Code,week,P,pop,incidence
0,A,2020-S20,15000,189581100,7.912181
1,A,2020-S21,18200,189581100,9.600113
2,A,2020-S22,16700,189581100,8.808895
3,A,2020-S23,8300,189581100,4.378074
4,A,2020-S24,6700,189581100,3.534108
...,...,...,...,...,...
2479,V,2022-S18,2425200,667111000,363.537702
2480,V,2022-S19,2041100,667111000,305.961077
2481,V,2022-S20,1536700,667111000,230.351471
2482,V,2022-S21,999600,667111000,149.840132


In [342]:
week_dep = pd.merge(week_dep,correspondence,on = 'Code',how = 'left')
incidence = week_dep

incidence.to_csv('/Users/chengyichen/Desktop/incidence.csv')

In [343]:
colnames = ["keywords"]
# Function pd.read_csv reads a comma-separated values (csv) file into DataFrame.
df = pd.read_excel("/Users/chengyichen/Desktop/keywords.xlsx")
df2 = df["keywords"].values.tolist()


In [336]:
# Dictionary of google regions with id and name 

regions_id_google = {'Nord_Pas_de_Calais':'O',
            'Picardie':'S',
            'Haute_Normandie':'Q',
            'Ile_de_France':'J',
            'Corse':'H',
            'PACA':'U',
            'Languedoc_Roussillon':'K',
            'Midi_Pyrénées':'N',
            'Aquitaine':'B',
            'Rhône_Alpes':'V',
            'Auvergne':'C', 
            'Limousin':'L',
            'Poitou_Charente':'T',
            'Basse_Normandie':'P',
            'Bretagne':'E',
            'Pays_de_la_Loire':'R',
            'Alsace':'A',
            'Lorraine':'M',
            'Champagne_Ardennes':'G',
            'Centre_Val_de_Loire':'F',
            'Bourgogne':'D',
            'Franche_Comté':'I'}


    
# Create dataframe for each region

region_df = regions_id_google.copy()
for reg in regions_id_google.keys():
    region_df[reg]=pd.DataFrame() 

In [344]:
trends = TrendReq(geo="FR")
output_dataset=pd.DataFrame()  
i=0

for region in regions_id_google.values():

    #retrieve region name (useful later for descriptive analysis)
    region_name = list(regions_id_google)[i]
    print(region, region_name)
    print('FR-'+region)  
    temp = []
    
    for x in range(0,len(df2)):
        keywords = [df2[x]]
        print(keywords)
        trends.build_payload(
        kw_list=keywords,
        cat=0,
        timeframe='2020-05-01 2022-05-31', 
        geo='FR-'+region)
        data = trends.interest_over_time()
        temp.append(data)


    region_df[reg]=pd.DataFrame (pd.concat(temp, axis=1))
    region_df[reg].insert(0, 'region_code', region) 
    region_df[reg].insert(1, 'region_name', region_name) 
    output_dataset=output_dataset.append(region_df[reg])
    print(output_dataset)
    i=i+1
    
output_dataset.to_csv('/Users/chengyichen/Desktop/search_trends.csv')    


O Nord_Pas_de_Calais
FR-O
['Covid-19']
['Coronavirus']
['Pandémie']
['Mesures sanitaires']
['Vaccin']
['Masque']
['Isolement']
['Quarantaine']
['Tests PCR']
['Tests antigéniques']
['Système immunitaire']
['Contagion']
['Virus']
['Santé publique']
['Restrictions']
['Couvre-feu']
['Confinement']
['Distanciation sociale']
['Symptômes']
['Fièvre']
['Toux']
['Essoufflement']
['Perte de goût']
["Perte d'odorat"]
['Fatigue']
['Douleurs musculaires']
['Maux de tête']
['Diarrhée']
['Gastro-intestinal']
['Respirateur artificiel']
['Soins intensifs']
['Mortalité']
['Vaccination']
['Effets secondaires']
['Efficacité']
['Variants']
['Delta']
['Alpha']
['Bêta']
['Gamma']
['Épidémie']
['Cluster']
["Taux d'infection"]
['Taux de positivité']
['Taux de mortalité']
['Immunité collective']
['Variant Delta plus']
['Pass sanitaire']
['Green Pass']
['Certificat sanitaire']
['Nouvelle vague']
['Protocoles sanitaires']
['Vaccinodrome']
['Vaccination de masse']
['Anxiété']
['Dépression']
['Impact économique']
[

/var/folders/w3/v_7zxn855qd1kj4wqy68swd80000gn/T/ipykernel_62955/2372316337.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  region_df[reg].insert(0, 'region_code', region)
/var/folders/w3/v_7zxn855qd1kj4wqy68swd80000gn/T/ipykernel_62955/2372316337.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  region_df[reg].insert(1, 'region_name', region_name)
/var/folders/w3/v_7zxn855qd1kj4wqy68swd80000gn/T/ipykernel_62955/2372316337.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

           region_code         region_name  Covid-19  isPartial  Coronavirus  \
date                                                                           
2020-05-03           O  Nord_Pas_de_Calais        21      False          100   
2020-05-10           O  Nord_Pas_de_Calais        21      False           86   
2020-05-17           O  Nord_Pas_de_Calais        19      False           66   
2020-05-24           O  Nord_Pas_de_Calais        18      False           54   
2020-05-31           O  Nord_Pas_de_Calais        18      False           40   
...                ...                 ...       ...        ...          ...   
2022-05-01           O  Nord_Pas_de_Calais         3      False            2   
2022-05-08           O  Nord_Pas_de_Calais         3      False            2   
2022-05-15           O  Nord_Pas_de_Calais         2      False            1   
2022-05-22           O  Nord_Pas_de_Calais         2      False            1   
2022-05-29           O  Nord_Pas_de_Cala

TooManyRequestsError: The request failed: Google returned a response with code 429

In [249]:
search_trends = pd.read_csv("/Users/chengyichen/Desktop/search_trends.csv")
search_trends

,date,Covid-19,Coronavirus,Pandémie,Mesures sanitaires,Vaccin,Masque,Isolement,Quarantaine,Tests PCR,...,Ventilation,Groupe à risque,Dépistage rapide,Auto-test,Vaccination pour les enfants,Stratégie vaccinale,Passeport vaccinal,Confinement national,Désinfection,Autotests COVID
0,2020-05-03,21,100,98,24,3,100,8,77,11,...,57,53,0,3,0,0,0,14,100,8
1,2020-05-10,20,89,100,25,4,69,7,62,7,...,63,0,0,0,8,0,0,5,89,0
2,2020-05-17,19,71,82,14,3,33,4,51,6,...,64,39,0,3,0,0,0,0,54,0
3,2020-05-24,17,53,53,23,3,21,7,41,5,...,75,19,0,0,8,0,1,2,51,8
4,2020-05-31,17,39,48,21,2,15,5,38,6,...,80,0,0,3,0,21,0,0,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2022-05-01,3,2,13,9,6,5,9,11,8,...,67,0,24,11,0,12,3,0,14,0
105,2022-05-08,3,2,15,10,5,8,8,8,10,...,74,22,14,11,12,0,3,0,17,5
106,2022-05-15,3,2,17,12,6,10,7,8,7,...,74,0,0,0,0,0,2,0,19,0
107,2022-05-22,3,1,19,5,6,5,5,12,10,...,62,0,0,10,0,0,4,0,12,0


In [311]:
pytrends = TrendReq()
pytrends.build_payload(['Euro 2021'], cat=0, timeframe='2020-05-01 2022-05-31',  gprop='', geo='FR')
df_index = pytrends.interest_over_time()
df_index = df_index.drop(columns=['isPartial'])

In [312]:
search_trends["date"] = pd.to_datetime(search_trends["date"])
df_index = pd.merge(df_index,search_trends, how = 'left',on = 'date')

df_index

,date,Euro 2021,Covid-19,Coronavirus,Pandémie,Mesures sanitaires,Vaccin,Masque,Isolement,Quarantaine,...,Ventilation,Groupe à risque,Dépistage rapide,Auto-test,Vaccination pour les enfants,Stratégie vaccinale,Passeport vaccinal,Confinement national,Désinfection,Autotests COVID
0,2020-05-03,0,21,100,98,24,3,100,8,77,...,57,53,0,3,0,0,0,14,100,8
1,2020-05-10,0,20,89,100,25,4,69,7,62,...,63,0,0,0,8,0,0,5,89,0
2,2020-05-17,0,19,71,82,14,3,33,4,51,...,64,39,0,3,0,0,0,0,54,0
3,2020-05-24,0,17,53,53,23,3,21,7,41,...,75,19,0,0,8,0,1,2,51,8
4,2020-05-31,0,17,39,48,21,2,15,5,38,...,80,0,0,3,0,21,0,0,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2022-05-01,0,3,2,13,9,6,5,9,11,...,67,0,24,11,0,12,3,0,14,0
105,2022-05-08,0,3,2,15,10,5,8,8,8,...,74,22,14,11,12,0,3,0,17,5
106,2022-05-15,0,3,2,17,12,6,10,7,8,...,74,0,0,0,0,0,2,0,19,0
107,2022-05-22,0,3,1,19,5,6,5,5,12,...,62,0,0,10,0,0,4,0,12,0


In [313]:
df_index = df_index[1:]
df_index = df_index.reset_index(drop=True)
df_index = df_index.reset_index()
df_index


,index,date,Euro 2021,Covid-19,Coronavirus,Pandémie,Mesures sanitaires,Vaccin,Masque,Isolement,...,Ventilation,Groupe à risque,Dépistage rapide,Auto-test,Vaccination pour les enfants,Stratégie vaccinale,Passeport vaccinal,Confinement national,Désinfection,Autotests COVID
0,0,2020-05-10,0,20,89,100,25,4,69,7,...,63,0,0,0,8,0,0,5,89,0
1,1,2020-05-17,0,19,71,82,14,3,33,4,...,64,39,0,3,0,0,0,0,54,0
2,2,2020-05-24,0,17,53,53,23,3,21,7,...,75,19,0,0,8,0,1,2,51,8
3,3,2020-05-31,0,17,39,48,21,2,15,5,...,80,0,0,3,0,21,0,0,41,0
4,4,2020-06-07,0,16,32,55,25,3,14,5,...,77,0,0,0,0,0,0,1,27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,103,2022-05-01,0,3,2,13,9,6,5,9,...,67,0,24,11,0,12,3,0,14,0
104,104,2022-05-08,0,3,2,15,10,5,8,8,...,74,22,14,11,12,0,3,0,17,5
105,105,2022-05-15,0,3,2,17,12,6,10,7,...,74,0,0,0,0,0,2,0,19,0
106,106,2022-05-22,0,3,1,19,5,6,5,5,...,62,0,0,10,0,0,4,0,12,0


In [314]:
all_incidence = pd.DataFrame(week_dep.groupby(['week'])['incidence'].mean())
all_incidence.columns
all_incidence = all_incidence.reset_index()
all_incidence = all_incidence.rename(columns={"week": "date"})
all_incidence = all_incidence.reset_index()
all_incidence

,index,date,incidence
0,0,2020-S20,4.367876
1,1,2020-S21,6.296901
2,2,2020-S22,5.157242
3,3,2020-S23,4.294993
4,4,2020-S24,4.510225
...,...,...,...
103,103,2022-S18,418.392326
104,104,2022-S19,338.624576
105,105,2022-S20,247.878168
106,106,2022-S21,177.224452


In [318]:
df_corr = pd.merge(df_index,all_incidence,on = "index",how = 'left')
df_corr = df_corr.set_index('index')
correlations = df_corr.corr()['incidence'].sort_values()
correlations = pd.DataFrame(abs(correlations).sort_values(ascending = False))
correlations[correlations['incidence']>0.6]

,incidence
incidence,1.000000
Auto-test,0.914843
Isolement,0.870648
Symptômes,0.814776
Contagion,0.753281
Fièvre,0.743743
Contact tracing,0.619361
Autotests COVID,0.610457


In [322]:
correlations[correlations['incidence']>0.6]
len(correlations[correlations['incidence']>0.2])
correlations[correlations['incidence']>0.2]

,incidence
incidence,1.000000
Auto-test,0.914843
Isolement,0.870648
Symptômes,0.814776
Contagion,0.753281
Fièvre,0.743743
Contact tracing,0.619361
Autotests COVID,0.610457
Essoufflement,0.542841
Maux de tête,0.534540


In [323]:
#We selected 40 keywords rank by correlation
keywords40 = ["Auto-test","Isolement","Symptômes","Contagion","Fièvre","Contact tracing","Autotests COVID","Essoufflement","Maux de tête","Toux",
              "Système immunitaire","Diarrhée","Urgences","Fatigue","Dépression","Hôpital","Variant omicron","Anxiété","Gamma","Soins à domicile",
              "Voyages","Green Pass","Isolement obligatoire","Télétravail","Coronavirus","Soins intensifs","Tests PCR","Tests antigéniques","Certificat sanitaire",
              "Ventilation","Perte de goût","Frontières","Quarantaine","Restrictions","Distanciation sociale","Alpha","Chômage","Gel hydroalcoolique","Réanimation","Aéroports"]

Final Data Set

In [334]:
data = pd.merge(incidence,region,on = "Code",how = 'left')
data = data.drop(columns=['Departement'])

,Code,week,P,pop,incidence,Departement_label,Region_google
0,A,2020-S20,15000,189581100,7.912181,Bas-Rhin,Alsace
1,A,2020-S20,15000,189581100,7.912181,Haut-Rhin,Alsace
2,A,2020-S21,18200,189581100,9.600113,Bas-Rhin,Alsace
3,A,2020-S21,18200,189581100,9.600113,Haut-Rhin,Alsace
4,A,2020-S22,16700,189581100,8.808895,Bas-Rhin,Alsace
...,...,...,...,...,...,...,...
10903,V,2022-S22,1271100,667111000,190.538006,Isere,Rhone-Alpes
10904,V,2022-S22,1271100,667111000,190.538006,Loire,Rhone-Alpes
10905,V,2022-S22,1271100,667111000,190.538006,Rhone,Rhone-Alpes
10906,V,2022-S22,1271100,667111000,190.538006,Savoie,Rhone-Alpes


In [345]:
merged_df = pd.read_csv('/Users/chengyichen/Desktop/merged_df.csv')
incidence = pd.read_csv('/Users/chengyichen/Desktop/incidence.csv')
incidence = incidence.rename(columns={"Code": "region"})
incidence = incidence.drop(columns=["Unnamed: 0"])
incidence 

,region,week,date,P,pop,incidence
0,A,2020-S20,2020-05-11,15000,189581100,7.912181
1,A,2020-S21,2020-05-18,18200,189581100,9.600113
2,A,2020-S22,2020-05-25,16700,189581100,8.808895
3,A,2020-S23,2020-06-01,8300,189581100,4.378074
4,A,2020-S24,2020-06-08,6700,189581100,3.534108
...,...,...,...,...,...,...
2479,V,2022-S18,2022-04-25,2425200,667111000,363.537702
2480,V,2022-S19,2022-05-02,2041100,667111000,305.961077
2481,V,2022-S20,2022-05-09,1536700,667111000,230.351471
2482,V,2022-S21,2022-05-16,999600,667111000,149.840132


In [346]:
import pandas as pd
from datetime import datetime, timedelta

merged_df['date'] = pd.to_datetime(merged_df['date'])  
merged_df['date'] = merged_df['date'] + timedelta(days=1)  
merged_df = merged_df.drop(columns=["Unnamed: 0"])
merged_df= merged_df[merged_df['date'] != '2020-05-04']
merged_df= merged_df[merged_df['date'] != '2022-05-30']
merged_df

,date,Auto-test,Isolement,Symptômes,Contagion,Fièvre,Contact tracing,Autotests COVID,Essoufflement,Maux de tête,...,Quarantaine,Restrictions,Distanciation sociale,Alpha,Chômage,Gel hydroalcoolique,Réanimation,Aéroports,region,region name
1,2020-05-11,36.0,9,41,7,86,0.0,0.0,0,17,...,0,0,0,47,100,92,61,22,M,Lorraine
2,2020-05-18,33.0,6,31,14,60,0.0,0.0,39,0,...,0,0,22,63,60,68,33,36,M,Lorraine
3,2020-05-25,37.0,14,34,26,41,100.0,75.0,23,69,...,51,0,0,62,62,48,0,23,M,Lorraine
4,2020-06-01,0.0,5,29,12,36,0.0,0.0,0,31,...,40,0,21,44,56,29,31,0,M,Lorraine
5,2020-06-08,28.0,6,23,23,21,0.0,0.0,0,0,...,61,18,0,66,50,19,38,0,M,Lorraine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2392,2022-04-25,0.0,10,61,25,15,0.0,0.0,0,0,...,0,0,0,49,41,0,15,0,A,Alsace
2393,2022-05-02,0.0,13,65,25,20,0.0,0.0,100,30,...,0,19,0,59,22,0,0,0,A,Alsace
2394,2022-05-09,38.0,0,57,23,49,0.0,0.0,57,63,...,12,14,0,36,9,9,0,62,A,Alsace
2395,2022-05-16,0.0,7,60,26,21,0.0,0.0,0,57,...,0,24,0,62,29,6,0,0,A,Alsace


In [348]:
import pandas as pd
import datetime


date_list = incidence['week']


new_date_list = []
for date_str in date_list:
    year, week_num = date_str.split("-S")
    week_num = int(week_num)

    first_day_of_year = datetime.datetime(int(year), 1, 1)
    first_day_of_week = first_day_of_year - datetime.timedelta(days=first_day_of_year.weekday())

    delta = datetime.timedelta(weeks=week_num-1)
    result_date = first_day_of_week + delta

    result_str = result_date.strftime("%Y-%m-%d")
    new_date_list.append(result_str)

incidence['date'] = new_date_list

In [349]:
cols = incidence.columns.tolist()
new_cols = ['region', 'week', 'date','P','pop','incidence']
cols_reordered = [c for c in new_cols if c in cols]
incidence = incidence.reindex(columns=cols_reordered)

print(incidence)
incidence.to_csv('/Users/chengyichen/Desktop/incidence.csv')

     region      week        date        P        pop   incidence
0         A  2020-S20  2020-05-11    15000  189581100    7.912181
1         A  2020-S21  2020-05-18    18200  189581100    9.600113
2         A  2020-S22  2020-05-25    16700  189581100    8.808895
3         A  2020-S23  2020-06-01     8300  189581100    4.378074
4         A  2020-S24  2020-06-08     6700  189581100    3.534108
...     ...       ...         ...      ...        ...         ...
2479      V  2022-S18  2022-04-25  2425200  667111000  363.537702
2480      V  2022-S19  2022-05-02  2041100  667111000  305.961077
2481      V  2022-S20  2022-05-09  1536700  667111000  230.351471
2482      V  2022-S21  2022-05-16   999600  667111000  149.840132
2483      V  2022-S22  2022-05-23  1271100  667111000  190.538006

[2484 rows x 6 columns]


In [350]:
merged_df['date'] = merged_df['date'].astype(str)
final= pd.merge(merged_df,incidence, on=['date', 'region'], how='inner') 
new_cols = list(final.columns[:1]) + list(final.columns[-6:]) + list(final.columns[1:-6])
final = final.reindex(columns=new_cols)
final.to_csv('/Users/chengyichen/Desktop/full_data_set.csv')

In [351]:
df_all = pd.read_csv('/Users/chengyichen/Desktop/full_data_set.csv')
df_all_gr = df_all.groupby(['region'])['pop','incidence'].mean()
df_all.drop(columns=["Unnamed: 0"])

/var/folders/w3/v_7zxn855qd1kj4wqy68swd80000gn/T/ipykernel_62955/3460328792.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_all_gr = df_all.groupby(['region'])['pop','incidence'].mean()


,date,region,region name,week,P,pop,incidence,Auto-test,Isolement,Symptômes,...,Perte de goût,Frontières,Quarantaine,Restrictions,Distanciation sociale,Alpha,Chômage,Gel hydroalcoolique,Réanimation,Aéroports
0,2020-05-11,M,Lorraine,2020-S20,24200,230742500,10.487882,36.0,9,41,...,0.0,51,0,0,0,47,100,92,61,22
1,2020-05-18,M,Lorraine,2020-S21,37800,230742500,16.381898,33.0,6,31,...,31.0,42,0,0,22,63,60,68,33,36
2,2020-05-25,M,Lorraine,2020-S22,18100,230742500,7.844242,37.0,14,34,...,0.0,86,51,0,0,62,62,48,0,23
3,2020-06-01,M,Lorraine,2020-S23,26700,230742500,11.571340,0.0,5,29,...,0.0,100,40,0,21,44,56,29,31,0
4,2020-06-08,M,Lorraine,2020-S24,23700,230742500,10.271190,28.0,6,23,...,0.0,93,61,18,0,66,50,19,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,2022-04-25,A,Alsace,2022-S18,821400,189581100,433.271038,0.0,10,61,...,24.0,16,0,0,0,49,41,0,15,0
2372,2022-05-02,A,Alsace,2022-S19,618700,189581100,326.351097,0.0,13,65,...,0.0,13,0,19,0,59,22,0,0,0
2373,2022-05-09,A,Alsace,2022-S20,386200,189581100,203.712290,38.0,0,57,...,15.0,25,12,14,0,36,9,9,0,62
2374,2022-05-16,A,Alsace,2022-S21,258700,189581100,136.458750,0.0,7,60,...,0.0,10,0,24,0,62,29,6,0,0


In [352]:
df_all_gr["pop"] = df_all_gr["pop"].apply(lambda x: '{:.2f}'.format(float(x)))
df_all_gr['pop'] = df_all_gr['pop'].astype(float) / 100
df_all_gr

,pop,incidence
region,,
A,1895811.0,413.710857
B,3467317.0,383.202685
C,1361267.0,371.690083
D,1612402.0,360.138454
E,2258306.0,338.080353
F,2559073.0,343.821333
G,1308511.0,366.800155
H,344679.0,392.310806
I,1170637.0,398.318276
